# Deep Learning Project - Gesture Recognition

## Problem Statement

Imagine you are working as a data scientist at a home electronics company which manufactures state of the art smart televisions. You want to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

# Gesture Recognition Feature for Smart TV

## Problem Statement
As part of our ongoing efforts to innovate in the home electronics space, our team at the cutting-edge smart television manufacturing company is embarking on developing an interactive feature for our smart TVs. This feature aims to recognize five distinct gestures performed by the user, enabling control of the TV without the necessity of a physical remote. This gesture-based control system promises to enhance user interaction by offering a more intuitive and seamless experience.

## Objectives

### Generator
- The generator is tasked with processing a batch of video inputs flawlessly. It must execute steps such as cropping, resizing, and normalization accurately, ensuring that the videos are properly formatted for model training and inference.

### Model Development
- The core of this project lies in developing a robust model capable of recognizing gestures with high accuracy while maintaining a minimal number of parameters. The efficiency of the model is paramount, as we aim to achieve low inference (prediction) times suitable for a responsive user interface on the smart TV.
  - **Initial Approach**: Begin with training the model on a smaller dataset to gauge its performance and identify potential areas for improvement.
  - **Progression Strategy**: Gradually increase the dataset size, fine-tuning and adjusting the model based on initial insights to enhance its accuracy and efficiency.

### Write-Up
- A comprehensive write-up will document the journey towards the final model selection. It will detail:
  - **Base Model Choice**: The rationale behind selecting the initial model as our starting point, considering its relevance and potential for the task at hand.
  - **Modifications and Experiments**: A chronicle of the modifications, experiments, and metrics considered throughout the project to refine and perfect the model.
  - **Final Model Selection**: The process and reasoning for choosing the final model, including discussions on the balance between model accuracy and the number of parameters, to ensure an optimal solution for real-time gesture recognition.

This project represents a significant leap towards creating more engaging and user-friendly smart TV experiences, leveraging advanced gesture recognition technology to set new standards in the industry.


#### Importing the Libraries

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os
from skimage.transform import resize
import imageio
import imageio.v2 as imageio

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

2024-02-03 10:09:14.531005: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /Users/smalagi/Desktop/Project_data**

In [3]:
train_doc = np.random.permutation(open('/Users/smalagi/Desktop/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/Users/smalagi/Desktop/Project_data/val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = resize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = resize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
        
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = '/Users/smalagi/Desktop/Project_data/train'
val_path = '/Users/smalagi/Desktop/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

#write your model here

#### Model a

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [7]:
#model a
model_a = Sequential()

model_a.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())

model_a.add(Dense(1000, activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(500, activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(5, activation='softmax'))

In [8]:
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001) # Updated to use `learning_rate` instead of `lr`
model_a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_a.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization (Batch  (None, 30, 120, 120, 8)   32        
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d (MaxPooling3  (None, 15, 60, 60, 8)     0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_1 (Bat  (None, 15, 60, 60, 16)    6

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

# Updated ModelCheckpoint to use `save_freq='epoch'`
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # ReduceLROnPlateau setup remains unchanged
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:

model_a.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
            callbacks=callbacks_list, validation_data=val_generator, 
            validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/2313827601.py:17: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/10
64/67 [===========================>..] - ETA: 14s - loss: 4.4204 - categorical_accuracy: 0.2922Batch:  67 Index: 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/2313827601.py:42: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 4.3440 - categorical_accuracy: 0.2911Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-4.34400-0.29110-1.30156-0.42000.h5
67/67 [==============================] - 362s 5s/step - loss: 4.3440 - categorical_accuracy: 0.2911 - val_loss: 1.3016 - val_categorical_accuracy: 0.4200 - lr: 0.0010
Epoch 2/10


/Users/smalagi/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


67/67 [==============================] - ETA: 0s - loss: 2.4085 - categorical_accuracy: 0.2836
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-2.40846-0.28358-1.31336-0.37000.h5
67/67 [==============================] - 147s 2s/step - loss: 2.4085 - categorical_accuracy: 0.2836 - val_loss: 1.3134 - val_categorical_accuracy: 0.3700 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.9519 - categorical_accuracy: 0.2985
Epoch 3: saving model to model_init_2024-02-0310_09_19.822402/model-00003-1.95192-0.29851-1.47133-0.31000.h5

Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
67/67 [==============================] - 153s 2s/step - loss: 1.9519 - categorical_accuracy: 0.2985 - val_loss: 1.4713 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 4/10
67/67 [==============================] - ETA: 0s - loss: 1.5154 - categorical_accuracy: 0.3383
Epoch 4: saving model to model_init_2024-02-0310_09_19.822402/mo

In [13]:
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
channel = 3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator_ex(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = resize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = resize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [14]:
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential

#### Model b

In [15]:
# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 30, 120, 120, 32   2624      
                             )                                   
                                                                 
 activation_4 (Activation)   (None, 30, 120, 120, 32   0         
                             )                                   
                                                                 
 conv3d_5 (Conv3D)           (None, 30, 120, 120, 32   27680     
                             )                                   
                                                                 
 activation_5 (Activation)   (None, 30, 120, 120, 32   0         
                             )                                   
                                                                 
 max_pooling3d_4 (MaxPoolin  (None, 10, 40, 40, 32)   

In [16]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [17]:
model_b.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/10
64/67 [===========================>..] - ETA: 1:13 - loss: 1.9284 - categorical_accuracy: 0.2047Batch:  67 Index: 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 1.9175 - categorical_accuracy: 0.2006 Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-1.91754-0.20060-1.60848-0.23000.h5
67/67 [==============================] - 1674s 25s/step - loss: 1.9175 - categorical_accuracy: 0.2006 - val_loss: 1.6085 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6109 - categorical_accuracy: 0.1891
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-1.61095-0.18905-1.60883-0.17000.h5
67/67 [==============================] - 562s 8s/step - loss: 1.6109 - categorical_accuracy: 0.1891 - val_loss: 1.6088 - val_categorical_accuracy: 0.1700 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6104 - categorical_accuracy: 0.1791
Epoch 3: saving model to model_init_2024-02-0310_09_19.822402/mod

In [18]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_8 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_9 (Conv3D)           (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_9 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_6 (MaxPoolin  (None, 10, 20, 20, 32)    0         
 g3D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [19]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [20]:
model_b.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/10
64/67 [===========================>..] - ETA: 17s - loss: 1.6371 - categorical_accuracy: 0.2172Batch:  67 Index: 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 1.6359 - categorical_accuracy: 0.2232Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-1.63590-0.22323-1.60568-0.24000.h5
67/67 [==============================] - 413s 6s/step - loss: 1.6359 - categorical_accuracy: 0.2232 - val_loss: 1.6057 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.5571 - categorical_accuracy: 0.2687
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-1.55708-0.26866-1.59051-0.25000.h5
67/67 [==============================] - 151s 2s/step - loss: 1.5571 - categorical_accuracy: 0.2687 - val_loss: 1.5905 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6391 - categorical_accuracy: 0.1940
Epoch 3: saving model to model_init_2024-02-0310_09_19.822402/model-

In [21]:
train_generator = generator_ex(train_path, train_doc, 20)
val_generator = generator_ex(val_path, val_doc, 20)

In [22]:
model_b.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 20


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/10
31/67 [============>.................] - ETA: 7:03 - loss: 1.0987 - categorical_accuracy: 0.5387Batch:  34 Index: 20


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 1.0858 - categorical_accuracy: 0.5420Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 20

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-1.08583-0.54199-1.21729-0.52000.h5
67/67 [==============================] - 515s 8s/step - loss: 1.0858 - categorical_accuracy: 0.5420 - val_loss: 1.2173 - val_categorical_accuracy: 0.5200 - lr: 5.0000e-04
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.0358 - categorical_accuracy: 0.5274
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-1.03581-0.52736-1.20644-0.46000.h5
67/67 [==============================] - 178s 3s/step - loss: 1.0358 - categorical_accuracy: 0.5274 - val_loss: 1.2064 - val_categorical_accuracy: 0.4600 - lr: 5.0000e-04
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.0570 - categorical_accuracy: 0.4776
Epoch 3: saving model to model_init_2024-02-0310_09_19.82240

In [23]:
train_generator = generator_ex(train_path, train_doc, 10)
val_generator = generator_ex(val_path, val_doc, 10)

In [24]:
model_b.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/10
64/67 [===========================>..] - ETA: 18s - loss: 0.5039 - categorical_accuracy: 0.8094Batch:  67 Index: 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 0.5089 - categorical_accuracy: 0.8069Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-0.50887-0.80694-1.23107-0.58000.h5
67/67 [==============================] - 434s 6s/step - loss: 0.5089 - categorical_accuracy: 0.8069 - val_loss: 1.2311 - val_categorical_accuracy: 0.5800 - lr: 6.2500e-05
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.4961 - categorical_accuracy: 0.8060
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-0.49611-0.80597-1.28238-0.53000.h5
67/67 [==============================] - 154s 2s/step - loss: 0.4961 - categorical_accuracy: 0.8060 - val_loss: 1.2824 - val_categorical_accuracy: 0.5300 - lr: 6.2500e-05
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.5608 - categorical_accuracy: 0.7761
Epoch 3: saving model to model_init_2024-02-0310_09_19.82240

In [25]:
train_generator = generator_ex(train_path, train_doc, 15)
val_generator = generator_ex(val_path, val_doc, 15)

In [26]:
model_b.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 15


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/10
42/67 [=================>............] - ETA: 3:24 - loss: 0.4480 - categorical_accuracy: 0.8397Batch:  45 Index: 15


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 0.4465 - categorical_accuracy: 0.8381Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 15
Batch:  7 Index: 15

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-0.44651-0.83813-1.24794-0.57692.h5
67/67 [==============================] - 443s 7s/step - loss: 0.4465 - categorical_accuracy: 0.8381 - val_loss: 1.2479 - val_categorical_accuracy: 0.5769 - lr: 7.8125e-06
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.4719 - categorical_accuracy: 0.8408
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-0.47191-0.84080-1.39794-0.52000.h5
67/67 [==============================] - 147s 2s/step - loss: 0.4719 - categorical_accuracy: 0.8408 - val_loss: 1.3979 - val_categorical_accuracy: 0.5200 - lr: 7.8125e-06
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.3771 - categorical_accuracy: 0.8955
Epoch 3: saving model to model_init_2024

In [27]:
model_b.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_8 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_9 (Conv3D)           (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_9 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_6 (MaxPoolin  (None, 10, 20, 20, 32)    0         
 g3D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [28]:
train_generator = generator_ex(train_path, train_doc, 16)
val_generator = generator_ex(val_path, val_doc, 16)

In [29]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/3161962229.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 16
Epoch 1/10
39/67 [================>.............] - ETA: 4:05 - loss: 0.4475 - categorical_accuracy: 0.8349Batch:  42 Index: 16


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 0.4401 - categorical_accuracy: 0.8413Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 16
Batch:  7 Index: 16

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-0.44005-0.84129-1.32737-0.56250.h5
67/67 [==============================] - 512s 8s/step - loss: 0.4401 - categorical_accuracy: 0.8413 - val_loss: 1.3274 - val_categorical_accuracy: 0.5625 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.4106 - categorical_accuracy: 0.8401
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-0.41064-0.84009-1.21635-0.57500.h5
67/67 [==============================] - 279s 4s/step - loss: 0.4106 - categorical_accuracy: 0.8401 - val_loss: 1.2163 - val_categorical_accuracy: 0.5750 - lr: 0.0010
Epoch 3/10
Batch:  95 Index: 7
67/67 [==============================] - ETA: 0s - loss: 0.4253 - categorical_accuracy: 0.8407
Epoch 3: saving model to mod

In [30]:
num_epochs = 18 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 18


In [31]:
import keras
model_b.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_8 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_9 (Conv3D)           (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_9 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_6 (MaxPoolin  (None, 10, 20, 20, 32)    0         
 g3D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [32]:
train_generator = generator_ex(train_path, train_doc, 18)
val_generator = generator_ex(val_path, val_doc, 18)

In [33]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/3161962229.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 18
Epoch 1/18
34/67 [==============>...............] - ETA: 5:33 - loss: 0.4047 - categorical_accuracy: 0.8382Batch:  37 Index: 18


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 0.4029 - categorical_accuracy: 0.8446Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 18
Batch:  6 Index: 18

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-0.40287-0.84456-1.33645-0.56429.h5
67/67 [==============================] - 673s 10s/step - loss: 0.4029 - categorical_accuracy: 0.8446 - val_loss: 1.3364 - val_categorical_accuracy: 0.5643 - lr: 0.0010
Epoch 2/18
67/67 [==============================] - ETA: 0s - loss: 0.4839 - categorical_accuracy: 0.8184
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-0.48391-0.81843-1.36662-0.59000.h5
67/67 [==============================] - 241s 4s/step - loss: 0.4839 - categorical_accuracy: 0.8184 - val_loss: 1.3666 - val_categorical_accuracy: 0.5900 - lr: 0.0010
Epoch 3/18
67/67 [==============================] - ETA: 0s - loss: 0.4311 - categorical_accuracy: 0.8308
Epoch 3: saving model to model_init_2024-02-031

#### Model c

In [34]:
model_c = Sequential()
model_c.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_c.add(Activation('relu'))
model_c.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Flatten())
model_c.add(Dense(512, activation='relu'))
model_c.add(BatchNormalization())
model_c.add(Dropout(0.5))
model_c.add(Dense(classes, activation='softmax'))
model_c.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_c.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_12 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_13 (Conv3D)          (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_13 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_8 (MaxPoolin  (None, 10, 20, 20, 32)    0         
 g3D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [35]:
# Changing back batch size, images per frame, height and width of image
batch_size = 10
x = 30 # number of frames
y = 120 # image width
z = 120# image height
num_epochs = 20

In [36]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

#### Model d

In [37]:
input_shape=(x,y,z,channel)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
# Define model
model_d = Sequential()

model_d.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_d.add(Flatten())

model_d.add(Dense(nb_dense[0], activation='relu'))
model_d.add(Dropout(0.5))

model_d.add(Dense(nb_dense[1], activation='relu'))
model_d.add(Dropout(0.5))

#softmax layer
model_d.add(Dense(nb_dense[2], activation='softmax'))
model_d.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_d.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_18 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_5 (Bat  (None, 30, 120, 120, 8)   32        
 chNormalization)                                                
                                                                 
 activation_18 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_11 (MaxPooli  (None, 15, 60, 60, 8)     0         
 ng3D)                                                           
                                                                 
 conv3d_19 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_6 (Bat  (None, 15, 60, 60, 16)   

In [38]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/3797531445.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 10
Epoch 1/10
64/67 [===========================>..] - ETA: 13s - loss: 3.2176 - categorical_accuracy: 0.2125Batch:  67 Index: 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 3.2095 - categorical_accuracy: 0.2081Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-3.20947-0.20814-1.61689-0.25000.h5
67/67 [==============================] - 337s 5s/step - loss: 3.2095 - categorical_accuracy: 0.2081 - val_loss: 1.6169 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.8139 - categorical_accuracy: 0.2786
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-2.81391-0.27861-1.62266-0.25000.h5
67/67 [==============================] - 139s 2s/step - loss: 2.8139 - categorical_accuracy: 0.2786 - val_loss: 1.6227 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 3.3033 - categorical_accuracy: 0.1791
Epoch 3: saving model to model_init_2024-02-0310_09_19.822402/model-

#### Model e

In [39]:
x = 30 # number of frames
y = 64 # image width
z = 64 # image height 

classes = 5
channel = 1

In [40]:
model_e = Sequential()
model_e.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_e.add(Activation('relu'))
model_e.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Flatten())
model_e.add(Dense(512, activation='relu'))
model_e.add(BatchNormalization())
model_e.add(Dropout(0.5))
model_e.add(Dense(classes, activation='softmax'))
model_e.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_e.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_22 (Conv3D)          (None, 30, 64, 64, 32)    896       
                                                                 
 activation_22 (Activation)  (None, 30, 64, 64, 32)    0         
                                                                 
 conv3d_23 (Conv3D)          (None, 30, 64, 64, 32)    27680     
                                                                 
 activation_23 (Activation)  (None, 30, 64, 64, 32)    0         
                                                                 
 max_pooling3d_15 (MaxPooli  (None, 10, 22, 22, 32)    0         
 ng3D)                                                           
                                                                 
 dropout_14 (Dropout)        (None, 10, 22, 22, 32)    0         
                                                      

In [41]:
batch_size = 5
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_e.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/1728685948.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_e.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 5
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 2.0358 - categorical_accuracy: 0.1940Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 5

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-2.03575-0.19403-1.60972-0.20000.h5
67/67 [==============================] - 235s 3s/step - loss: 2.0358 - categorical_accuracy: 0.1940 - val_loss: 1.6097 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 2/10
63/67 [===========================>..] - ETA: 12s - loss: 2.0301 - categorical_accuracy: 0.1810Batch:  133 Index: 5


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 2.0452 - categorical_accuracy: 0.1813
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-2.04515-0.18127-1.61515-0.16000.h5
67/67 [==============================] - 230s 3s/step - loss: 2.0452 - categorical_accuracy: 0.1813 - val_loss: 1.6152 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 2.0531 - categorical_accuracy: 0.1741
Epoch 3: saving model to model_init_2024-02-0310_09_19.822402/model-00003-2.05314-0.17413-1.61569-0.18000.h5

Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
67/67 [==============================] - 147s 2s/step - loss: 2.0531 - categorical_accuracy: 0.1741 - val_loss: 1.6157 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 4/10
67/67 [==============================] - ETA: 0s - loss: 1.9087 - categorical_accuracy: 0.2189
Epoch 4: saving model to model_init_2024-02-0310_09_19.822402/mo

#### Model f

In [42]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 1
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_f = Sequential()

model_f.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_f.add(Flatten())

model_f.add(Dense(nb_dense[0], activation='relu'))
model_f.add(Dropout(0.5))

model_f.add(Dense(nb_dense[1], activation='relu'))
model_f.add(Dropout(0.5))

#softmax layer
model_f.add(Dense(nb_dense[2], activation='softmax'))
model_f.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_f.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_28 (Conv3D)          (None, 30, 120, 120, 8)   224       
                                                                 
 batch_normalization_10 (Ba  (None, 30, 120, 120, 8)   32        
 tchNormalization)                                               
                                                                 
 activation_28 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_18 (MaxPooli  (None, 15, 60, 60, 8)     0         
 ng3D)                                                           
                                                                 
 conv3d_29 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_11 (Ba  (None, 15, 60, 60, 16)   

In [43]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/3462510936.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/10
64/67 [===========================>..] - ETA: 13s - loss: 3.4070 - categorical_accuracy: 0.1703Batch:  67 Index: 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 3.4024 - categorical_accuracy: 0.1719Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-3.40237-0.17195-1.60182-0.31000.h5
67/67 [==============================] - 345s 5s/step - loss: 3.4024 - categorical_accuracy: 0.1719 - val_loss: 1.6018 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 3.2698 - categorical_accuracy: 0.2040
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-3.26983-0.20398-1.60892-0.30000.h5
67/67 [==============================] - 142s 2s/step - loss: 3.2698 - categorical_accuracy: 0.2040 - val_loss: 1.6089 - val_categorical_accuracy: 0.3000 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 3.2391 - categorical_accuracy: 0.1891
Epoch 3: saving model to model_init_2024-02-0310_09_19.822402/model-

#### Model g

In [44]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_g = Sequential()

model_g.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_g.add(Flatten())

model_g.add(Dense(nb_dense[0], activation='relu'))
model_g.add(Dropout(0.5))

model_g.add(Dense(nb_dense[1], activation='relu'))
model_g.add(Dropout(0.5))

#softmax layer
model_g.add(Dense(nb_dense[2], activation='softmax'))
model_g.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_g.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_32 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_14 (Ba  (None, 30, 120, 120, 8)   32        
 tchNormalization)                                               
                                                                 
 activation_32 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_22 (MaxPooli  (None, 15, 60, 60, 8)     0         
 ng3D)                                                           
                                                                 
 conv3d_33 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_15 (Ba  (None, 15, 60, 60, 16)   

In [45]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_g.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/2187855109.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_g.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/10
64/67 [===========================>..] - ETA: 12s - loss: 3.0206 - categorical_accuracy: 0.1859Batch:  67 Index: 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 3.0366 - categorical_accuracy: 0.1840Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-3.03656-0.18401-1.60694-0.28000.h5
67/67 [==============================] - 330s 5s/step - loss: 3.0366 - categorical_accuracy: 0.1840 - val_loss: 1.6069 - val_categorical_accuracy: 0.2800 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.8482 - categorical_accuracy: 0.1990
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-2.84817-0.19900-1.60961-0.24000.h5
67/67 [==============================] - 142s 2s/step - loss: 2.8482 - categorical_accuracy: 0.1990 - val_loss: 1.6096 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 2.7310 - categorical_accuracy: 0.2139
Epoch 3: saving model to model_init_2024-02-0310_09_19.822402/model-

#### Model h

In [46]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
model_h = Sequential()

model_h.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_h.add(Flatten())

model_h.add(Dense(nb_dense[0], activation='relu'))
model_h.add(Dropout(0.5))

model_h.add(Dense(nb_dense[1], activation='relu'))
model_h.add(Dropout(0.5))

#softmax layer
model_h.add(Dense(nb_dense[2], activation='softmax'))
model_h.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_h.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_36 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_18 (Ba  (None, 30, 120, 120, 8)   32        
 tchNormalization)                                               
                                                                 
 activation_36 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_26 (MaxPooli  (None, 15, 60, 60, 8)     0         
 ng3D)                                                           
                                                                 
 conv3d_37 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_19 (Ba  (None, 15, 60, 60, 16)   

#### Model final

In [47]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

# Define model
model_final = Sequential()

model_final.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(Activation('relu'))
model_final.add(Dropout(0.25))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_final.add(Flatten())

model_final.add(Dense(nb_dense[0], activation='relu'))
model_final.add(Dropout(0.5))

model_final.add(Dense(nb_dense[1], activation='relu'))
model_final.add(Dropout(0.5))

#softmax layer
model_final.add(Dense(nb_dense[2], activation='softmax'))

In [48]:
optimiser = tf.keras.optimizers.Adam() #write your optimizer
model_final.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_final.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_40 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_22 (Ba  (None, 30, 120, 120, 8)   32        
 tchNormalization)                                               
                                                                 
 activation_40 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_30 (MaxPooli  (None, 15, 60, 60, 8)     0         
 ng3D)                                                           
                                                                 
 conv3d_41 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_23 (Ba  (None, 15, 60, 60, 16)   

In [49]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [50]:
batch_size = 10
num_epochs = 12
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Users/smalagi/Desktop/Project_data/train ; batch size = 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/655006215.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/12
64/67 [===========================>..] - ETA: 12s - loss: 2.0349 - categorical_accuracy: 0.2547Batch:  67 Index: 10


/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 2.0226 - categorical_accuracy: 0.2489Source path =  /Users/smalagi/Desktop/Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-2.02263-0.24887-1.54481-0.43000.h5
67/67 [==============================] - 327s 5s/step - loss: 2.0226 - categorical_accuracy: 0.2489 - val_loss: 1.5448 - val_categorical_accuracy: 0.4300 - lr: 0.0010
Epoch 2/12
67/67 [==============================] - ETA: 0s - loss: 1.5048 - categorical_accuracy: 0.3134
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-1.50479-0.31343-1.62629-0.18000.h5
67/67 [==============================] - 137s 2s/step - loss: 1.5048 - categorical_accuracy: 0.3134 - val_loss: 1.6263 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 3/12
67/67 [==============================] - ETA: 0s - loss: 1.5532 - categorical_accuracy: 0.2736
Epoch 3: saving model to model_init_2024-02-0310_09_19.822402/model-

In [51]:
batch_size = 12
num_epochs = 14
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/3909543589.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/var/folders/v6/6bvfh58j79sf_23ln412v_fr0000gq/T/ipykernel_26161/157304017.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/14
67/67 [==============================] - ETA: 0s - loss: 1.0905 - categorical_accuracy: 0.5672
Epoch 1: saving model to model_init_2024-02-0310_09_19.822402/model-00001-1.09050-0.56716-0.99309-0.64000.h5
67/67 [==============================] - 137s 2s/step - loss: 1.0905 - categorical_accuracy: 0.5672 - val_loss: 0.9931 - val_categorical_accuracy: 0.6400 - lr: 2.5000e-04
Epoch 2/14
67/67 [==============================] - ETA: 0s - loss: 1.0675 - categorical_accuracy: 0.5721
Epoch 2: saving model to model_init_2024-02-0310_09_19.822402/model-00002-1.06747-0.57214-0.98272-0.58000.h5
67/67 [==============================] - 135s 2s/step - loss: 1.0675 - categorical_accuracy: 0.5721 - val_loss: 0.9827 - val_categorical_accuracy: 0.5800 - lr: 2.5000e-04
Epoch 3/14
67/67 [==============================] - ETA: 0s - loss: 1.0469 - categorical_accuracy: 0.5721
Epoch 3: saving model to model_init_2024-02-0310_09_19.822402/model-00003-1.04687-0.57214-0.89766-0.68000.h5
67/67 [======

## Best Model Selection
The model from Epoch 14 is identified as the best-fit model for deployment, highlighted by its exceptional performance metrics:

- **Highest Validation Accuracy:**
  - Achieved a validation accuracy of 80.00%, indicating a strong ability of the model to generalize well to new, unseen data.

- **Lowest Validation Loss:**
  - Recorded a validation loss of 0.6323, suggesting the model has a good understanding of the data with minimal error in its predictions.

## Conclusion
Based on the comprehensive evaluation, the model trained up to Epoch 14 (`model-00014-0.72762-0.75622-0.63232-0.80000.h5`) is highly recommended for deployment. This recommendation is based on several key factors:

- Provides the highest validation accuracy among all models evaluated, underlining its superior capability to correctly interpret and classify gesture data.
- Demonstrates effective learning and generalization capability, crucial for real-world applications where the model will encounter diverse and previously unseen gesture data.
- Strikes an optimal balance between learning efficiency and performance, considering the inherent complexity and variability of gesture data. This balance ensures that the model is not only accurate but also practical for real-time applications in gesture recognition.

This model represents a significant step forward in developing intuitive and user-friendly interaction methods for smart TV applications, leveraging the latest advancements in 3D convolutional neural networks and machine learning.
